In [10]:
import os 
import json
import numpy as np
import pandas as pd
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

from sklearn.pipeline import make_pipeline

import warnings
#warnings.filterwarnings("ignore")

In [11]:
!ls /mnt/C89CF3A00D398BCA/Users/HOME/Programming/Entrance\ tasks/Cinimex\ Data\ Lab/data

student-mat.csv  student-merge.R  student-por.csv  student.txt


In [7]:
df = pd.read_csv("/data/student-mat.csv", ";")
df.head()

FileNotFoundError: File b'/data/student-mat.csv' does not exist

In [11]:
numeric = ["age", 'Medu', "Fedu", "traveltime", "studytime", "failures",\
           "famrel", "freetime", "goout", "Dalc", "Walc", "health", "absences", "G1", "G2"]

X, y = df[numeric], df["G3"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42)

In [15]:
#making pipeline
pipe = make_pipeline(RandomForestRegressor())
pipe

Pipeline(steps=[('randomforestregressor', RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False))])

In [38]:
#setting up grid search
t = RandomForestRegressor()
print(t.get_params().keys())
param_grid = {"randomforestregressor__n_estimators" : [10, 20, 30],
             "randomforestregressor__max_depth" : [None, 6, 8, 10],
             "randomforestregressor__max_leaf_nodes": [None, 5, 10, 20], 
             "randomforestregressor__min_impurity_split": [0.1, 0.2, 0.3]}

#running grid search
grid = GridSearchCV(pipe, param_grid=param_grid, cv=3)

grid.fit(X_train, y_train)

dict_keys(['bootstrap', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])


GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(steps=[('randomforestregressor', RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'randomforestregressor__n_estimators': [10, 20, 30], 'randomforestregressor__max_depth': [None, 6, 8, 10], 'randomforestregressor__max_leaf_nodes': [None, 5, 10, 20], 'randomforestregressor__min_impurity_split': [0.1, 0.2, 0.3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [39]:
print("Best parameters: {}".format(grid.best_params_))
print()
print("Validation set score: {:.2f}".format(grid.score(X_test, y_test)))

Best parameters: {'randomforestregressor__max_depth': 6, 'randomforestregressor__max_leaf_nodes': None, 'randomforestregressor__min_impurity_split': 0.2, 'randomforestregressor__n_estimators': 30}

Validation set score: 0.76


In [40]:
#saving model
import dill as pickle
filename = 'model_v1.pk'

with open('flask_api/models/' + filename, 'wb') as file:
    pickle.dump(grid, file)

test_df = X_test
with open("flask_api/models/model_v1.pk", "rb") as file:
    loaded_mdoel = pickle.load(file)
loaded_mdoel.predict(test_df)

array([  7.65771791,  11.75536448,   6.8279828 ,   9.21527288,
         9.21960747,  12.74024116,  18.43117025,   7.36828981,
         7.50369813,  13.68939447,  15.53908385,   7.27364417,
        14.13919495,  11.57698879,  14.13947566,   8.10173326,
         1.35914299,  10.78393929,  15.2749301 ,   4.39619957,
        13.89660097,  15.3947856 ,  15.1313949 ,   6.5337037 ,
         6.92644074,  18.96553773,  10.9002691 ,  10.03307091,
        17.87701681,  10.92557405,   9.07976301,   8.66348207,
        15.1618546 ,  12.97659658,   1.651237  ,   6.54941138,
         0.        ,  15.30237821,  11.78838443,   7.92424414])

In [45]:
# testing server
import json
import requests

"""Setting the headers to send and accept json responses
"""
header = {'Content-Type': 'application/json', \
                  'Accept': 'application/json'}

"""Reading test batch
"""
#df = pd.read_csv('../data/test.csv', encoding="utf-8-sig")
#df = df.head()

"""Converting Pandas Dataframe to json
"""
data = test_df.to_json(orient='records')

In [52]:
"""POST <url>/predict
"""
#link = """https://deploying-predictions-tutorial.herokuapp.com/predict"""
link = """http://0.0.0.0:8007/predict"""

resp = requests.post(link, \
                    data = json.dumps(data),\
                    headers= header)

In [53]:
resp.status_code

200

In [54]:
"""The final response we get is as follows:
"""
resp.json()

{'predictions': '[{"0":5.7765337903},{"0":5.6044252044},{"0":5.7998220298},{"0":4.9475787638},{"0":5.6563564214},{"0":5.6044252044},{"0":5.9551112314},{"0":5.9960575998},{"0":1.4},{"0":4.8649657287},{"0":5.8620556758},{"0":5.9923538961},{"0":5.5664622415},{"0":5.4775733526},{"0":5.7218326118},{"0":1.8},{"0":5.9024597162},{"0":5.8320893458},{"0":4.5554834055},{"0":5.8654226792},{"0":4.5892328042},{"0":4.4804834055},{"0":5.8695893458},{"0":6.0783387446},{"0":5.780700457},{"0":5.9923538961},{"0":5.6044252044},{"0":4.731758658},{"0":5.9139075277},{"0":1.2},{"0":6.0783387446},{"0":5.4822029822},{"0":5.5155363155},{"0":5.8168783069},{"0":5.6044252044},{"0":5.6563564214},{"0":5.817737494},{"0":6.0725186388},{"0":5.5525733526},{"0":5.7588696489}]'}

In [58]:
resp.json()["predictions"]

'[{"0":5.7765337903},{"0":5.6044252044},{"0":5.7998220298},{"0":4.9475787638},{"0":5.6563564214},{"0":5.6044252044},{"0":5.9551112314},{"0":5.9960575998},{"0":1.4},{"0":4.8649657287},{"0":5.8620556758},{"0":5.9923538961},{"0":5.5664622415},{"0":5.4775733526},{"0":5.7218326118},{"0":1.8},{"0":5.9024597162},{"0":5.8320893458},{"0":4.5554834055},{"0":5.8654226792},{"0":4.5892328042},{"0":4.4804834055},{"0":5.8695893458},{"0":6.0783387446},{"0":5.780700457},{"0":5.9923538961},{"0":5.6044252044},{"0":4.731758658},{"0":5.9139075277},{"0":1.2},{"0":6.0783387446},{"0":5.4822029822},{"0":5.5155363155},{"0":5.8168783069},{"0":5.6044252044},{"0":5.6563564214},{"0":5.817737494},{"0":6.0725186388},{"0":5.5525733526},{"0":5.7588696489}]'

In [59]:
pd.read_json(resp.json())

ValueError: Invalid file path or buffer object type: <class 'dict'>

In [63]:
from pandas.io.json import json_normalize
json_normalize(resp.json(), sep=",")

,predictions
0,"[{""0"":5.7765337903},{""0"":5.6044252044},{""0"":5...."


In [51]:
t = loaded_mdoel.predict(test_df)
pd.DataFrame(t)

,0
0,7.657718
1,11.755364
2,6.827983
3,9.215273
4,9.219607
5,12.740241
6,18.431170
7,7.368290
8,7.503698
9,13.689394
